In [6]:
import data,  models, vizz

import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [7]:
# temp code for dev
import importlib
importlib.reload(data)
importlib.reload(models)
importlib.reload(vizz)

<module 'vizz' from 'c:\\Users\\rache\\OneDrive\\Documents\\GitHub\\kaggle-disaster-tweets\\src\\vizz.py'>

In [8]:
df_train = data.get_data('../data/raw/train.csv')
df_test = data.get_data('../data/raw/test.csv')

In [9]:
#train doc2vec model
model_doc2vec = models.train_doc2vec(df_train, vector_size=10, min_count=1, epochs=20)

In [10]:
X = df_train[['keyword','location','text']]
y = df_train[['target']]

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.3)

X_train_vectors = data.preprocess_documents(X_train)
X_val_vectors = data.preprocess_documents(X_val)

clf = RandomForestClassifier()
clf.fit(X_train_vectors,np.ravel(y_train))

y_pred = clf.predict(X_val_vectors)
accuracy_score(y_val, y_pred)

0.7088441330998249